# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import datetime as dt

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

import pickle
import re



# Load and Clean

## Load Data

In [2]:
df_raw = pd.read_csv('HMXPC13_DI_v2_5-14-14.csv')

In [3]:
df_raw = df_raw.drop('roles', axis = 1)

## Drop Unnecessary Columns

In [6]:
#These columns contain useless information, or info that is not applicable to my model
#Grade and last_event are too predictive, and would not be known explicitly until the 
#end of the course

#dropping nevents because it is colinear with nplay_video

df_raw_edx = df_raw.drop(columns=['registered', 'viewed', 'grade', 'last_event_DI', 'incomplete_flag', 'nevents'])

### Make Mask, check if user has either explored or been certified, Then create new DF

In [10]:
mask = (df_raw_edx['explored'] == 1) | (df_raw_edx['certified'] == 1)
df_edx = df_raw_edx[mask]

## Clean Nans

In [13]:
df_edx.groupby(['course_id']).agg({'nplay_video': 'sum'})

,nplay_video
course_id,
HarvardX/CB22x/2013_Spring,0.0
HarvardX/CS50x/2012,0.0
HarvardX/ER22x/2013_Spring,0.0
HarvardX/PH207x/2012_Fall,3191567.0
HarvardX/PH278x/2013_Spring,216255.0
MITx/14.73x/2013_Spring,1049757.0
MITx/2.01x/2013_Spring,183007.0
MITx/3.091x/2012_Fall,611536.0
MITx/3.091x/2013_Spring,106349.0


In [14]:
#Create masks for each id
mask = df_edx['course_id'] != 'HarvardX/CB22x/2013_Spring' 
mask2 = df_edx['course_id'] != 'HarvardX/CS50x/2012' 
mask3 = df_edx['course_id'] != 'HarvardX/ER22x/2013_Spring'

In [15]:
#drop and filter each consecutive mask, store in df
first = df_edx[mask]
second = first[mask2]
third = second[mask3]

/Users/evanisenstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/evanisenstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [16]:
df = third

### Replace LoE_DI, gender nans with none provided 
This data can either be considered nans, or the question was asked
before the survey was asked

In [18]:
df[['LoE_DI', 'gender']] = df[['LoE_DI','gender']].fillna('None Provided')

/Users/evanisenstein/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


### Change Yob to column median

In [19]:
yob_median = df['YoB'].median()
df['YoB'].replace({np.nan: yob_median}, inplace = True)

/Users/evanisenstein/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


### Drop leftover nans, should still be large enough of a dataset to work with

In [21]:
df.dropna(inplace=True)

/Users/evanisenstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## Replace column names and get dummies

### Basic Feature Engineering, create column based on season (fall, spring, or summer)

In [24]:
df['season'] = 'spring'

/Users/evanisenstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
list(df['course_id'].unique())

['HarvardX/PH207x/2012_Fall',
 'HarvardX/PH278x/2013_Spring',
 'MITx/3.091x/2012_Fall',
 'MITx/6.002x/2012_Fall',
 'MITx/7.00x/2013_Spring',
 'MITx/6.00x/2012_Fall',
 'MITx/6.00x/2013_Spring',
 'MITx/6.002x/2013_Spring',
 'MITx/14.73x/2013_Spring',
 'MITx/8.02x/2013_Spring',
 'MITx/8.MReV/2013_Summer',
 'MITx/3.091x/2013_Spring',
 'MITx/2.01x/2013_Spring']

In [26]:
fall_dates = ['HarvardX/PH207x/2012_Fall','MITx/3.091x/2012_Fall',
              'MITx/6.002x/2012_Fall','MITx/6.00x/2012_Fall']

In [27]:
mask = df['course_id'] == 'HarvardX/PH207x/2012_Fall'
df.loc[mask, 'season'] = 'fall'

mask = df['course_id'] == 'MITx/3.091x/2012_Fall'
df.loc[mask, 'season'] = 'fall'

mask = df['course_id'] == 'MITx/6.002x/2012_Fall'
df.loc[mask, 'season'] = 'fall'

mask = df['course_id'] == 'MITx/6.00x/2012_Fall'
df.loc[mask, 'season'] = 'fall'

mask = df['course_id'] == 'MITx/8.MReV/2013_Summer'
df.loc[mask, 'season'] = 'summer'

/Users/evanisenstein/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Relative Launch
Subtract Course Launch from actual start time, depends on the progrm

In [29]:
#convert start_time_DI into datetime format
df['start_time_DI'] = pd.to_datetime(df['start_time_DI'])

/Users/evanisenstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### Set Baseline for relative_launch, 
Healthstat, PH207x

In [30]:
df['relative_launch'] = (df['start_time_DI'] - dt.datetime(2012, 10, 15)).dt.days

/Users/evanisenstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


#### Use relative launch to create relative launch date

In [31]:
mask = df['course_id'] == 'HarvardX/PH278x/2013_Spring'
df.loc[mask, 'relative_launch'] = (df['start_time_DI'] - dt.datetime(2013, 5, 15)).dt.days

In [33]:
def change_relative_launch(course_id, y, m, d):
    mask = df['course_id'] == course_id
    df.loc[mask, 'relative_launch'] = (df['start_time_DI'] - dt.datetime(y, m, d)).dt.days

In [34]:
# 'MITx/3.091x/2012_Fall', 
change_relative_launch('MITx/3.091x/2012_Fall', 2012, 10, 9)


In [35]:
change_relative_launch('MITx/6.002x/2012_Fall', 2012, 9, 5)

In [36]:
change_relative_launch('MITx/7.00x/2013_Spring', 2013, 3, 3)

In [37]:
change_relative_launch('MITx/6.00x/2012_Fall', 2012, 9, 26)

In [38]:
change_relative_launch('MITx/6.00x/2013_Spring', 2013, 2, 4)

In [39]:
change_relative_launch('MITx/6.002x/2013_Spring', 2013, 3, 3)

In [40]:
change_relative_launch('MITx/14.73x/2013_Spring', 2012, 2, 12)

In [41]:
change_relative_launch('MITx/8.02x/2013_Spring', 2013, 2, 18)

In [42]:
change_relative_launch('MITx/8.MReV/2013_Summer', 2013, 6, 1)

In [43]:
change_relative_launch('MITx/3.091x/2013_Spring', 2013, 2, 5)

In [44]:
change_relative_launch('MITx/2.01x/2013_Spring', 2013, 4, 15)

## Replace identical course ids with single id, get better understanding of courses

In [46]:
df = df.replace({'course_id': {'MITx/6.00x/2012_Fall': 'MITx_CS_Python',
                            'MITx/6.00x/2013_Spring': 'MITx_CS_Python',
                            'MITx/6.002x/2012_Fall': 'MITx_Circuits',
                            'MITx/6.002x/2013_Spring': 'MITx_Circuits',
                            'MITx/3.091x/2012_Fall': 'MITx_SSChem',
                            'MITx/3.091x/2013_Spring': 'MITx_SSChem',
                            'HarvardX/PH207x/2012_Fall': 'Harv_HealthStat',
                            'HarvardX/PH278x/2013_Spring': 'Harv_HealthEnv',
                            'MITx/7.00x/2013_Spring': 'MITx_Biology',
                            'MITx/14.73x/2013_Spring': 'MITx_Poverty',
                            'MITx/8.02x/2013_Spring': 'MITx_ElecMag',
                            'MITx/8.MReV/2013_Summer': 'MITx_MechRev',
                            'MITx/2.01x/2013_Spring': 'MITx_Structures'}})

In [47]:
df['course_id'].unique()

array(['Harv_HealthStat', 'Harv_HealthEnv', 'MITx_SSChem',
       'MITx_Circuits', 'MITx_Biology', 'MITx_CS_Python', 'MITx_Poverty',
       'MITx_ElecMag', 'MITx_MechRev', 'MITx_Structures'], dtype=object)

## Running Get_Dummies
Get Dummies on course_id, final_cc_cname_DI, LoE_DI, YoB, gender, 'season'

In [49]:
dum_df = pd.get_dummies(data = df, columns=['course_id', 'final_cc_cname_DI', 'LoE_DI',
                                           'gender', 'season'])

In [50]:
dum_df.shape

(23959, 66)

# Logistic Regression with Dummies


## Dropping columns

Need to Drop   userid has to many individuals to be a useful feature and start_time because of how relative it is, thats why i created a relative start date

In [51]:
second_mvp_df = dum_df.drop(columns=['userid_DI', 'start_time_DI'])
second_mvp_df.reset_index(drop=True, inplace=True)                                

I am assigning the certified column to y, then I will be dropping it since it is easier to load the rest of the df into the X variable

In [52]:
y = second_mvp_df.loc[:, 'certified']

In [53]:
second_mvp_df.drop(columns = 'certified', inplace = True)

## Dropping columns that are too predictive, to make it a better model

In [54]:
#Dropped nchapters just to see what happens
second_mvp_df.drop(columns = ['nchapters', 'ndays_act', 'nplay_video', 'nforum_posts'], inplace=True)

In [55]:
X = second_mvp_df.iloc[:,:]

In [56]:
#second_mvp_df.head()
X.head()

,explored,YoB,relative_launch,course_id_Harv_HealthEnv,course_id_Harv_HealthStat,course_id_MITx_Biology,course_id_MITx_CS_Python,course_id_MITx_Circuits,course_id_MITx_ElecMag,course_id_MITx_MechRev,...,LoE_DI_Less than Secondary,LoE_DI_Master's,LoE_DI_None Provided,LoE_DI_Secondary,gender_None Provided,gender_f,gender_m,season_fall,season_spring,season_summer
0,1,1987.0,-28,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
1,1,1987.0,-76,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
2,1,1987.0,-57,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
3,1,1987.0,-60,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
4,1,1987.0,-53,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0


## Train Test Split

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    test_size = .25,
                                                    random_state = 42)

## Logistic Regression CV

In [58]:
std = StandardScaler()
logCV = LogisticRegressionCV(cv=5, random_state=75,
                             multi_class='multinomial')

#Fit and Transform based on X_train values
std.fit(X_train.values)
X_tr = std.transform(X_train.values)

#Transform X_test values based on the X_train fit
X_te = std.transform(X_test.values)

#Fit model using the X_tr values against the y_train 
#Cross validation is already built in to the method
logCV.fit(X_tr, y_train)

#use trained model to pred y values on unseen data set, save as y_pred
y_pred = logCV.predict(X_te)

In [59]:
logCV.C_

array([0.00599484])

In [60]:
f'The Logistic Regression CV F1 score is {f1_score(y_test, y_pred):.4f}'

'The Logistic Regression CV F1 score is 0.7113'

In [61]:
f'The Logistic Regression CV Recall is {recall_score(y_test, y_pred)}'

'The Logistic Regression CV Recall is 0.7850916741809438'

In [62]:
f'The Logistic Regression CV Precision score is {precision_score(y_test, y_pred):.4f}'

'The Logistic Regression CV Precision score is 0.6502'

## Intepretation

In [63]:
logCV.coef_

array([[-1.61181534e-01,  6.33940783e-02, -6.10827528e-01,
        -8.82867778e-02, -1.68430029e-01, -6.89002573e-02,
        -1.14670232e-01, -2.92090574e-02, -9.78381178e-02,
         1.12654028e-02,  6.05093288e-01,  1.05866112e-02,
        -7.01955297e-02,  3.90799909e-03, -2.29844113e-02,
        -2.28395713e-02, -6.57148199e-03, -1.09986988e-02,
        -2.49374476e-03, -2.19078086e-02, -6.15061521e-03,
         2.14885477e-02,  1.18893870e-02, -2.45030184e-02,
         1.77153694e-04, -1.61264331e-02,  4.30857702e-03,
        -1.30545538e-02, -1.21706383e-02, -4.29317136e-02,
        -1.94627971e-02,  2.00262841e-02, -2.13179010e-02,
         7.26429014e-03,  2.99499000e-03, -1.68488795e-02,
        -1.63164375e-02, -7.35646033e-03, -1.96312501e-02,
         4.91336211e-02,  1.35439712e-02,  3.12165801e-02,
         5.18633910e-02, -7.82765590e-03,  1.73152603e-02,
         8.77302871e-03, -2.80008812e-02, -1.57389435e-02,
         2.12902738e-03, -5.37595833e-03,  2.55941691e-0

In [64]:
interp_df = pd.DataFrame(data = logCV.coef_, columns = X.columns)

In [65]:
df_coef = interp_df.T.sort_values(by=0, ascending=False)

In [66]:
df_coef.head()

,0
course_id_MITx_Poverty,0.605093
YoB,0.063394
final_cc_cname_DI_Spain,0.051863
final_cc_cname_DI_Poland,0.049134
final_cc_cname_DI_Russian Federation,0.031217


In [67]:
interp_df.T.sort_values(by=0)

,0
relative_launch,-0.610828
course_id_Harv_HealthStat,-0.168430
explored,-0.161182
course_id_MITx_CS_Python,-0.114670
course_id_MITx_ElecMag,-0.097838
course_id_Harv_HealthEnv,-0.088287
course_id_MITx_Structures,-0.070196
course_id_MITx_Biology,-0.068900
final_cc_cname_DI_Other Africa,-0.042932
course_id_MITx_Circuits,-0.029209
